In [ ]:
from dotenv import load_dotenv
from metagame import TradingClient
import os

load_dotenv()

client = TradingClient(
    api_url=os.environ["API_URL"],
    jwt=os.environ["JWT"],
    act_as=int(os.environ["ACT_AS"]),
)

In [8]:
from IPython.display import clear_output

import collections
import pandas as pd
import pprint

def get_position_by_name(markets):
    state = client.state()

    for market in markets:
        market_id = state.market_name_to_id.get(market)    
        if market_id is None:
            return pd.DataFrame()

        if not state.markets[market_id].hasFullTradeHistory:
            client.get_full_trade_history(market_id)
            state = client.state()

        trades = state.markets[market_id].trades

        position_sums = collections.defaultdict(lambda: {
            "buy_size": 0.0,
            "buy_cost": 0.0,
            "sell_size": 0.0,
            "sell_cost": 0.0,
        })
        
        for t in trades:
            buyer = state.accounts[t.buyer_id].name
            seller = state.accounts[t.seller_id].name

            position_sums[buyer]["buy_size"] += t.size
            position_sums[buyer]["buy_cost"] += t.size * t.price

            position_sums[seller]["sell_size"] += t.size
            position_sums[seller]["sell_cost"] += t.size * t.price

        results = {}
        for user_id, sums in position_sums.items():
            buy_size = sums["buy_size"]
            buy_cost = sums["buy_cost"]
            sell_size = sums["sell_size"]
            sell_cost = sums["sell_cost"]
            
            avg_buy_price = buy_cost / buy_size if buy_size != 0 else 0.0
            avg_sell_price = sell_cost / sell_size if sell_size != 0 else 0.0
            
            results[user_id] = {
                "bought_volume": buy_size,
                "sold_volume": sell_size,
                "avg_buy_price": avg_buy_price,
                "avg_sell_price": avg_sell_price,
            }
        print(market)
        pprint.pp(results)

try:
    while True:
        get_position_by_name(["min_sat_test","max_sat_test", "sum_sat_test"])
        client.wait_portfolio_update()
        clear_output(wait=True)
except KeyboardInterrupt:
    clear_output(wait=True)
get_position_by_name(["min_sat_test"])

min_sat_test
{'sat_mark': {'bought_volume': 13.089999999999998,
              'sold_volume': 0.13999999999999999,
              'avg_buy_price': 6.975935828877004,
              'avg_sell_price': 4.5357142857142865},
 'sat_Grass': {'bought_volume': 0.0,
               'sold_volume': 11.01,
               'avg_buy_price': 0.0,
               'avg_sell_price': 7.40599455040872},
 'sat_bob': {'bought_volume': 0.15,
             'sold_volume': 2.089999999999998,
             'avg_buy_price': 4.500000000000001,
             'avg_sell_price': 4.696172248803832}}
